<a href="https://colab.research.google.com/github/zaraaa12/BigData/blob/main/Case_9_Data_Cleaning_with_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Install PySpark
!pip install pyspark

In [ ]:
# 2. Initialize Spark Session
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lower, regexp_replace, to_date, when
spark = SparkSession.builder.appName ("DataCleaningExercise" ).getOrCreate ()

In [ ]:
# 3. Reading a CSV File
df = spark.read.csv( '/content/dataset/data_transaksi.csv', header=True, inferSchema=True)
df.show (100)

+----------+--------------------+-----+----------+-----------------+----------+---------------+
|      Nama|               Email| Umur|Pendapatan|Tanggal Pembelian|    Produk|Nilai Transaksi|
+----------+--------------------+-----+----------+-----------------+----------+---------------+
|   Anthony|   dawnday@gmail.com| 24.0|   3553265|       01/13/1974|    Laptop|        2736049|
|    Robert|    pbrown@gmail.com| NULL|   5525753|       03-05-2004|    Tablet|        4996241|
|   Melissa| ethan99@clayton.com| NULL|   4911936|       28-04-1970|  Handpone|        3169456|
|   Timothy|johnsonmegan@gmai...|200.0|3140485039|       03/09/2003|    Laptop|         506622|
|     Kelly|keithwebster(at)e...|200.0|3833611071|       17-01-2022|   Tablett|        9969204|
|     Jared| qwiggins@hodges.com| 69.0|   8024480|       02-01-2005|   Tablett|        3093351|
|    Nicole|ogonzalez@hotmail...|200.0|4286346096|       08-01-2024| Handphone|        3007158|
|     Mason|lisasmith@hotmail...| NULL| 

In [ ]:
# 4. Exploring Data
df.printSchema ()           # Column Structure
df.describe ( ).show ()     # Numerical Statistics
df.columns                  # List of Columns

root
 |-- Nama: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- Umur: double (nullable = true)
 |-- Pendapatan: long (nullable = true)
 |-- Tanggal Pembelian: string (nullable = true)
 |-- Produk: string (nullable = true)
 |-- Nilai Transaksi: integer (nullable = true)

+-------+------+-------------------+------------------+--------------------+-----------------+---------+------------------+
|summary|  Nama|              Email|              Umur|          Pendapatan|Tanggal Pembelian|   Produk|   Nilai Transaksi|
+-------+------+-------------------+------------------+--------------------+-----------------+---------+------------------+
|  count|   100|                100|                68|                 100|              100|      100|               100|
|   mean|  NULL|               NULL|111.86764705882354|     1.65132351882E9|             NULL|     NULL|        5556026.09|
| stddev|  NULL|               NULL| 77.42110175763524|1.8103144429023266E9|             N

['Nama',
 'Email',
 'Umur',
 'Pendapatan',
 'Tanggal Pembelian',
 'Produk',
 'Nilai Transaksi']

In [ ]:
# 5. Delete Duplicate Rows
df_clean = df.dropDuplicates()

# 6. Delete Rows with Empty Values in the Important Column
df_clean = df_clean.na.drop(subset=["Nama", "Email"])

# 7. Filling Missing Value
df_clean = df_clean.fillna({'Umur': 0})

In [ ]:
# 8. Standardize Text Formatting: ‘Nama’ Field to All Lowercase Letters
df_clean = df_clean.withColumn("Nama", lower(col("Nama")))

In [ ]:
# 9. Change the Data Type ‘Umur’ to Integer
df_clean = df_clean.withColumn("Umur", col("Umur").cast("int"))

In [ ]:
# 10. Correct the Spelling in the Product Column:
#    Example: "Tablett" → "Tablet", "Handpone" → "Handphone", "Smartwach" → "Smartwatch"
df_clean = df_clean.withColumn(
    "Produk",
    when(col("Produk") == "Tablett", "Tablet")
    .when(col("Produk") == "Handpone", "Handphone")
    .when(col("Produk") == "Smartwach", "Smartwatch")
    .otherwise(col("Produk"))
)

In [ ]:
# 11. Date Format Standardization (Date of Purchase):
#     Handle formats like "dd-MM-yyyy" and "MM/dd/yyyy"
#     Detect mixed formats and convert them to yyyy-MM-dd
df_clean = df_clean.withColumn(
    "Tanggal Pembelian",
    when(col("Tanggal Pembelian").rlike(r"\d{2}/\d{2}/\d{4}"),
         to_date(col("Tanggal Pembelian"), "MM/dd/yyyy"))
    .when(col("Tanggal Pembelian").rlike(r"\d{2}-\d{2}-\d{4}"),
         to_date(col("Tanggal Pembelian"), "dd-MM-yyyy"))
    .otherwise(None)
)

In [ ]:
# 12. Display the Final Result for Checking
df_clean.show(100)

+----------+--------------------+----+----------+-----------------+----------+---------------+
|      Nama|               Email|Umur|Pendapatan|Tanggal Pembelian|    Produk|Nilai Transaksi|
+----------+--------------------+----+----------+-----------------+----------+---------------+
|   timothy|johnsonmegan@gmai...| 200|3140485039|       2003-03-09|    Laptop|         506622|
|      mike|owensrobert@hotma...|  59|3741985672|       2004-11-28|    Laptop|        7711290|
|  madeline|martinezmario@gma...| 200|3200189219|       1979-12-05|Smartwatch|        1908029|
|   raymond|santiagoscott(at)...| 200|2866365700|       1993-07-18|Smartwatch|        6530558|
|  benjamin|brittanygreen@mar...| 200|4075982395|       1982-02-17|    Tablet|        9067792|
|    leslie|maysannette@gmail...| 200|   8372468|       1981-09-13|Smartwatch|        4957082|
|   cynthia|lonnieandrews@cis...|  49|   5139026|       2000-02-10|    Tablet|        2393089|
|    debbie| allison19@yahoo.com|  41|4974434132| 

In [ ]:
# 13. Save the Clean Data Results to a New File
df_clean.write.csv("/content/dataset/data_transaksi_bersih.csv", header=True, mode='overwrite')